In [ ]:
import json
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

In [ ]:
SCOPES = ['https://mail.google.com/']
email = 'jackson.shinpaugh@gmail.com'

CREDENTIALS = json.loads(os.environ['CREDENTIALS'])
GMAIL_TOKEN = json.loads(os.environ['GMAIL_TOKEN'])

In [ ]:
def get_credentials():
    creds = Credentials.from_authorized_user_info(GMAIL_TOKEN, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS, SCOPES)
            creds = flow.run_local_server(port=0)
    return creds

In [ ]:
def get_senders(service):
    messages = service.users().messages().list(userId='me').execute()

    senders = []
    for message in messages['messages']:
        message_id = message['id']
        message = service.users().messages().get(userId='me', id=message_id, format='metadata', metadataHeaders='From').execute()
        headers = message['payload']['headers']
        senders.append(headers[0]['value'])
    
    return list(set(senders))

In [ ]:
def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages

In [ ]:
def delete_messages(service, query):
    messages_to_delete = search_messages(service, query)
    # it's possible to delete a single message with the delete API, like this:
    # service.users().messages().delete(userId='me', id=msg['id'])
    # but it's also possible to delete all the selected messages with one query, batchDelete
    return service.users().messages().batchDelete(
      userId='me',
      body={
          'ids': [ msg['id'] for msg in messages_to_delete]
      }
    ).execute()

In [ ]:
creds = get_credentials()
service = build('gmail', 'v1', credentials=creds)

In [ ]:
senders = get_senders(service)
senders

In [ ]:
messages = search_messages(service, 'Spotify')
len(messages)

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
mess = list(chunks(messages, 1000))
print(len(mess))

In [ ]:
response = delete_messages(service, 'Greek Theatre')